<a href="https://colab.research.google.com/github/AstraBert/SacCerML/blob/main/SacCerML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [38]:
!python3 -m pip install biopython==1.81

In [3]:
from Bio import BiopythonWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', BiopythonWarning)
    from Bio.SeqUtils.ProtParam import ProteinAnalysis
    from Bio.SeqUtils.CheckSum import crc32
    from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex
    from Bio.SeqUtils.CodonUsageIndices import SharpEcoliIndex
    from Bio.SeqUtils import six_frame_translations
    from Bio.Seq import Seq
    from Bio import SeqIO
import gzip
from math import floor


In [5]:
## CSV structure: ORF_TYPE,CAI,CHECKSUM,HIDROPHOBICITY,ISOELECTRIC,AROMATIC,INSTABLE,MW,HELIX,TURN,SHEET,MOL_EXT_RED,MOL_EXT_OX
def load_data(infile):
    """Load data from infile if it is in fasta format (after having unzipped it, if it is zipped)"""
    if infile.endswith(".gz"):  # If file is gzipped, unzip it
        y = gzip.open(infile, "rt", encoding="latin-1")
        # Read file as fasta if it is fasta
        if infile.endswith(".fasta.gz") or infile.endswith(".fna.gz") or infile.endswith(".fas.gz") or infile.endswith(".fa.gz"):
            records = SeqIO.parse(y, "fasta")
            sequences = {}
            for record in records:
                sequences.update({str(record.id): str(record.seq)})
            y.close()
            return sequences
        else:
            y.close()
            raise ValueError("File is the wrong format")
    # Read file directly as fasta if it is a not zipped fasta: handle also more uncommon extensions :-)
    elif infile.endswith(".fasta") or infile.endswith(".fna") or infile.endswith(".fas") or infile.endswith(".fa"):
        with open(infile, "r") as y:
            records = SeqIO.parse(y, "fasta")
            sequences = {}
            for record in records:
                sequences.update({str(record.id): str(record.seq)})
            y.close()
            return sequences
    else:
        raise ValueError("File is the wrong format")

In [6]:
def calculate_cai(dna,index=SharpEcoliIndex):
    cai = CodonAdaptationIndex()
    cai.set_cai_index(index)
    if len(dna) % 3 == 0:
        a = cai.cai_for_gene(dna)
    else:
        six_translated = six_frame_translations(dna)
        n = six_translated.split("\n")
        frames = {"0;F": n[5], "1;F": n[6], "2;F": n[7],"0;R": n[12], "1;R": n[11], "2;R": n[10]}
        ind = 0
        for i in list(frames.keys()):
            k = frames[i].replace(" ","")
            if "M" in k and "*" in k:
                if i.split(";")[0]=="F" and k.index("M") < k.index("*"):
                    if len(k) <= len(dna)/3:
                        ind = int(i.split("")[0])
                        break
                elif i.split(";")[0]=="R" and k.index("M") > k.index("*"):
                    if len(k) <= len(dna)/3:
                        ind = len(dna) - int(i.split("")[0])
                        break
        if ind == 0:
            cods = 3*floor(len(dna)/3)
            dna = dna[:cods]
            a = cai.cai_for_gene(dna)
        elif 1 <= ind <= 2:
            if len(dna[ind:])%3==0:
                dna = dna[ind:]
            else:
                cods = 3*floor((len(dna)-ind)/3)
                dna = dna[ind:cods+ind]
                a = cai.cai_for_gene(dna)
        else:
            if len(dna[:ind])%3==0:
                dna = dna[ind:]
            else:
                cods = 3*floor((len(dna)-ind)/3)
                dna = dna[:cods]
                a = cai.cai_for_gene(dna)
    return a


def checksum(dna):
    return crc32(dna)


def hidrophobicity(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    hydrophobicity_score = ProteinAnalysis(protein_sequence).gravy()
    return hydrophobicity_score

def isoelectric_pt(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    isoelectric = ProteinAnalysis(protein_sequence).isoelectric_point()
    return isoelectric

def aromatic(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    arom = ProteinAnalysis(protein_sequence).aromaticity()
    return arom


def instable(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    inst = ProteinAnalysis(protein_sequence).instability_index()
    return inst

def weight(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    wgt = ProteinAnalysis(protein_sequence).molecular_weight()
    return wgt

def sec_struct(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    second_struct = ProteinAnalysis(protein_sequence).secondary_structure_fraction()
    return ",".join([str(s) for s in second_struct])

def mol_ext(dna):
    protein_sequence = str(Seq(dna).translate())
    protein_sequence = protein_sequence.replace("*","")
    molar_ext = ProteinAnalysis(protein_sequence).molar_extinction_coefficient()
    return ",".join([str(s) for s in molar_ext])


In [7]:
!python3 -m pip install orfipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.7 MB/s eta 0:00:00
  Created wheel for orfipy: filename=orfipy-0.0.4-cp310-cp310-linux_x86_64.whl size=551358 sha256=b1e7f616b68f68ed375ece8ce0e78d2048d99f2e31f9bf25156e42baada24898
  Stored in directory: /root/.cache/pip/wheels/95/09/ab/cdd01e13be3152391ec0439bbffaa2c870e8f3badc88674b56
Successfully built orfipy


In [33]:
from orfipy_core import orfs
import sys

def longest_orf(coding):
    keys_M_starting = [key for key in list(coding.keys()) if str(Seq(coding[key]).translate()).startswith("M")]
    M_starting = [seq for seq in list(coding.values()) if str(Seq(seq).translate()).startswith("M")]
    lengths = [len(seq) for seq in M_starting]
    max_ind = lengths.index(max(lengths))
    return {keys_M_starting[max_ind]: M_starting[max_ind]}


def predict_orf(seq,minlen=45,maxlen=18000,longest_M_starting_orf_only=True):
    ls = orfs(seq,minlen=minlen,maxlen=maxlen)
    coding = {}
    count = 0
    for start,stop,strand,description in ls:
        count+=1
        if start < stop:
            coding.update({f"ORF.{count}": seq[int(start):int(stop)]})
        else:
            coding.update({f"ORF.{count}": str(Seq(seq[int(stop):int(start)]).reverse_complement())})
    if longest_M_starting_orf_only:
        warnings.warn("\n---------------------------\nWarning: option longest_M_starting_orf_only is set to True and thus you will get only the longest M-starting ORF; to get all the ORFs, set it to False\n---------------------------\n")
        return longest_orf(coding)
    return coding

In [34]:
def process_dna(fasta_file, longest_M_starting_orf_only):
    fas = load_data(fasta_file)
    seqs = [seq for seq in list(fas.values())]
    heads = [seq for seq in list(fas.keys())]
    data = {}
    proteins = {}
    for i in range(len(seqs)):
        coding = predict_orf(seqs[i],longest_M_starting_orf_only=longest_M_starting_orf_only)
        open_reading_frames = list(coding.keys())
        for key in open_reading_frames:
            head = f"{heads[i]}.{key}"
            proteins.update({head: str(Seq(coding[key]).translate())})
            cai = calculate_cai(coding[key])
            cksm = checksum(coding[key])
            hydr = hidrophobicity(coding[key])
            isl = isoelectric_pt(coding[key])
            arm = aromatic(coding[key])
            inst = instable(coding[key])
            mw = weight(coding[key])
            se_st = sec_struct(coding[key]).split(",")
            se_st1 = se_st[0]
            se_st2 = se_st[1]
            se_st3 = se_st[2]
            me = mol_ext(coding[key]).split(",")
            me1 = me[0]
            me2 = me[1]
            n = pd.DataFrame({"CAI": [cai], "CHECKSUM": [cksm], "HIDROPHOBICITY": [hydr], "ISOELECTRIC": [isl],"AROMATIC": [arm],"INSTABLE": [inst], "MW": [mw], "HELIX": [se_st1], "TURN": [se_st2], "SHEET": [se_st3],"MOL_EXT_RED": [me1], "MOL_EXT_OX": [me2]})
            data.update({head: n})
    return data, proteins


In the next cell, you will have to load the following files from the data folder:
- scerevisiae.csv
- test.fasta

In [11]:
from google.colab import files

csv = files.upload()

Saving scerevisiae.csv to scerevisiae.csv
Saving test.fasta to test.fasta


In [12]:
print("Loading data...")
# Load the data from the CSV file
data = pd.read_csv('scerevisiae.csv')
print("Loaded data")

Loading data...
Loaded data


In [13]:
print("Generating training and test data...")
# Features
X = data.iloc[:, 1:]

# Labels
y = data['ORF_TYPE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Generated training and test data")

Generating training and test data...
Generated training and test data


In [14]:
print("Building and training the model...")
# Create and train the Decision Tree classifier
model = DecisionTreeClassifier()

Building and training the model...


In [36]:
model = model.fit(X, y)
print("Built and trained the model... Now predicting")

Built and trained the model... Now predicting


In [16]:
from sklearn.metrics import accuracy_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9992559523809523


In [28]:
def predict_genes(infile, longest_M_starting_orf_only, model=model):
    X, proteins = process_dna(infile,longest_M_starting_orf_only)
    headers = list(X.keys())
    predictions = []
    for x in list(X.values()):
      p = model.predict(x)
      predictions.append(p)
    for i in range(len(predictions)):
        print(f"{headers[i]} protein sequence is\n{proteins[headers[i]]}\nand is predicted as {predictions[i][0]}\n")

In [39]:
from Bio import BiopythonWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', BiopythonWarning)
    warnings.simplefilter('ignore', UserWarning)
    inf = "test.fasta"
    predict_genes(inf, True)

YGR292w.ORF.7 protein sequence is
MTISDHPETEPKWWKEATIYQIYPASFKDSNNDGWGDLKGITSKLQYIKDLGVDAIWVCPFYDSPQQDMGYDISNYEKVWPTYGTNEDCFELIDKTHKLGMKFITDLVINHCSTEHEWFKESRSSKTNPKRDWFFWRPPKGYDAEGKPIPPNNWKSFFGGSAWTFDETTNEFYLRLFASRQVDLNWENEDCRRAIFESAVGFWLDHGVDGFRIDTAGLYSKRPGLPDSPIFDKTSKLQHPNWGSHNGPRIHEYHQELHRFMKNRVKDGREIMTVGEVAHGSDNALYTSAARYEVSEVFSFTHVEVGTSPFFRYNIVPFTLKQWKEAIASNFLFINGTDSWATTYIENHDQARSITRFADDSPKYRKISGKLLTLLECSLTGTLYVYQGQEIGQINFKEWPIEKYEDVDVKNNYEIIKKSFGKNSKEMKDFFKGIALLSRDHSRTPMPWTKDKPNAGFTGPDVKPWFLLNESFEQGINVEQESRDDDSVLNFWKRALQARKKYKELMIYGYDFQFIDLDSDQIFSFTKEYEDKTLFAALNFSGEEIEFSLPREGASLSFILGNYDDTDVSSRVLKPWEGRIYLVK
and is predicted as Verified_ORF

